In [250]:
import json
from haralyzer import HarParser, HarPage
from bs4 import BeautifulSoup
import pandas as pd
import os

# filename = input('Enter the name of the file: ')
# in the HAR Files folder, get the names of all the files
files = os.listdir('./HAR Files')


In [251]:

def get_schedules(data,filename):
    classDF = pd.DataFrame(columns=['Class Name','Unique', 'Day + Hour', 'Room', 'Instruction Mode', 'Instructor','Status', 'Flags', 'Core'])

    for i in range (len(data['entries'])):
        html_content = data['entries'][i]['response']['content']['text']
        soup = BeautifulSoup(html_content, 'html.parser')
        tables = soup.find_all('table')
        for table in tables:
            class_name = ''
            # Find all the rows in the current table
            rows = table.find_all('tr')
            
            # Iterate through the rows
            for i,row in enumerate(rows):
                # Find all the cells in the current row
                # print(row, '\n')
                cells = row.find_all(['th','td'])

                
                # Process and print cell data
                header = False
                rowList = []
                days =[]
                hours = []
                for j,cell in enumerate(cells):
                    if 'course_header' in cell.get('class', []):
                        
                        class_name = cell.get_text()
                        
                        header = True
                    else:
                        if(j == 1 or j == 2):
                            spans = cell.find_all('span')
                            
                            # print(spans)


                            # check if 

                            for i in range(len(spans)):
                                if j == 1:
                                    days.append(spans[i].get_text())
                                else:
                                    hours.append(spans[i].get_text())
                        # print(key)

                        else:
                            value = cell.get_text(strip=True)

                            rowList.append(value)

                        # if we detect a header, we don't want to print a row
                
                
                
                if header == False and i>0:
                    # print(class_name)
                    rowList.insert(0, class_name)
                    if(len(days)!= len(hours)):
                        print(len(days),len(hours))
                        print('days', days)
                        print('hours', hours)
                    rowList.insert(2, tuple(zip(tuple(days), tuple(hours))))

                    classDF.loc[len(classDF)] =  rowList
                    rowList = []
            

                    

                    # check the ending if cropping data!!!
                    
                    # classDF.loc[len(classDF)] =  classifications
                    
            # print(days, hours)

    classDF['Day + Hour'].apply(tuple)
    classDF.drop_duplicates(inplace=True)
    classDF.to_csv(f'./CSVs/{filename}.csv', index=False)

In [252]:
for file in files:
    print(file)
    if file == '.DS_Store':
        continue
    with open(f'./HAR Files/{file}', 'r') as f: 
        har_parser = HarParser(json.loads(f.read()))
    data = har_parser.har_data
    get_schedules(data,file[:-4])
    f.close()

TWTh.har
MTWTh.har
MTh.har
MWF.har
TTh.har
TThF.har
.DS_Store
ThF.har
Th.har
S.har
F.har
WF.har
WTh.har
T.har
W.har
TW.har
MT.har
M.har
MWTh.har
MW.har
TF.har
MTWThF.har
ThFS.har
MF.har
MTW.har
FS.har
MTTh.har


In [253]:
file[:-4]

'MTTh'

In [254]:
# go through each <tr>. Assign the first course-header class to the current course name
# check if the next <tr> is a course-header class. If so, assign it to the current course name
# if not, then convert the <tr> to a row in the table, and add the course name to that row


In [255]:
# go through every csv in the csvs folder, adn combine them into one csv
import glob
import pandas as pd
df = pd.concat(map(pd.read_csv, glob.glob('csvs/*.csv'))).drop_duplicates()
df.to_csv('Ut Course Catalog.csv', index=False)